In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from astropy.stats import mad_std, sigma_clip
import re
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
from cytoolz import get_in, partial
import astropy.stats
from astropy.time import Time
from datetime import datetime, timedelta
from pathlib import Path
import pickle

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.projects.sigma_circuits.segmentation as segmentation
import paulssonlab.projects.sigma_circuits.matriarch_stub as matriarch_stub
import paulssonlab.projects.sigma_circuits.experiment as experiment
import paulssonlab.io.metadata as metadata

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
hv.extension("bokeh")

# Load data

In [ ]:
# nd2_filename = "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/210324/PWM_flipped.nd2"
# nd2_filename2 = "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/210204/psc101_bistable.nd2"

In [ ]:
# nd2 = matriarch_stub.get_nd2_reader(nd2_filename)

In [ ]:
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329_expt210324flipped.pickle"
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329_expt210204nonflipped_psc101.pickle"
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210402_expt210331ramp.pickle"
filename = "/n/groups/paulsson/jqs1/sigma-circuits/210507_expt210505.pickle"
data = pickle.load(open(filename, "rb"))
df = data["table"]
md = data["metadata"]
mux_log = data["mux_log"]
experiment_txt = data["experiment.txt"]
grid_df = data["grid"]

# Acquisition times

In [ ]:
md.keys()

In [ ]:
dtimeabsolute = get_in(
    ("SLxPictureMetadata", "dTimeAbsolute"),
    md["image_metadata_sequence"],
)
dtimeabsolute = Time(dtimeabsolute, format="jd").to_datetime()
dtimemsec = get_in(
    ("SLxPictureMetadata", "dTimeMSec"),
    md["image_metadata_sequence"],
)
tdelta = timedelta(milliseconds=dtimemsec)

In [ ]:
tdelta.total_seconds()

In [ ]:
first_position_times = md["acquisition_times"][:: len(grid_df)] / 1e3  # seconds

In [ ]:
last_good_cycle = np.argmax(np.diff(first_position_times) == 0) - 1

In [ ]:
seconds_per_timepoint = first_position_times[last_good_cycle] / last_good_cycle

In [ ]:
mux_times = [m[0] for m in mux_log]

In [ ]:
mux_timepoints = [
    (m - dtimeabsolute).total_seconds() / seconds_per_timepoint for m in mux_times
]

In [ ]:
mux_timepoints

In [ ]:
mux_timepoints2 = [
    np.argmax((m - dtimeabsolute).total_seconds() <= first_position_times)
    for m in mux_times
]

In [ ]:
mux_timepoints2

In [ ]:
plt.plot(md["acquisition_times"][:5000])

# Filtered

In [ ]:
sensor_size_md = get_in(
    (
        "variant",
        "no_name",
        "vGrabberCameraSettings",
        "FormatFast",
        "fmtDesc",
        "sizeSensorPixels",
    ),
    md["grabber_settings"],
)
sensor_width = int(get_in(("cx", "@value"), sensor_size_md))
sensor_height = int(get_in(("cy", "@value"), sensor_size_md))

In [ ]:
%%time
# skip background
cells = df.loc[IDX[:, :, 1:]].join(grid_df).reset_index()
cells["dist_from_center"] = np.sqrt(
    (cells["centroid-0"] - sensor_width) ** 2
    + (cells["centroid-1"] - sensor_height) ** 2
)
filtered_cells = cells[
    cells["area"].between(200, 800)
    & cells["GFP-PENTA"].between(100, 4000)
    & cells["RFP-PENTA"].between(475, 4000)
    & ~cells["row"].isin(["F"])
    & cells["dist_from_center"]
    < 3000
][["GFP-PENTA", "RFP-PENTA", "area", "row", "pos", "t"]]

In [ ]:
cells["dist_from_center"].hvplot.hist()

In [ ]:
cells["RFP-PENTA"][cells["RFP-PENTA"].between(100, 3e3)].hvplot.hist(bins=300)

In [ ]:
cells["RFP-PENTA"][
    (cells["RFP-PENTA"] > 1) & ~np.isnan(cells["RFP-PENTA"])
].hvplot.hist(logy=True)

In [ ]:
%%time
groupby = "row"  # row or pos

medians = filtered_cells.groupby([groupby, "t"]).agg(
    ["median", astropy.stats.median_absolute_deviation]
)


def get_limits(x):
    x = x.droplevel(0, axis=1)
    return pd.DataFrame(
        {
            "lower": x["median"] - x["median_absolute_deviation"],
            "upper": x["median"] + x["median_absolute_deviation"],
        }
    )


limits = medians.groupby(level=0, axis=1).apply(get_limits)

In [ ]:
observable = "GFP-PENTA"

if groupby == "pos":
    medians2 = medians[observable][["median"]].join(grid_df)
    limits2 = limits[observable].join(grid_df)
    mask = medians2["row"].isin(["A"])  # TODO
    medians2 = medians2[mask]
    limits2 = limits2[mask]
else:
    medians2 = medians[observable].reset_index()
    limits2 = limits[observable].reset_index()
mean_plot = medians2.hvplot.line("t", "median", by=groupby, logy=True)
noise_plot = limits2.hvplot.area(
    x="t", y="lower", y2="upper", by=groupby, stacked=False, alpha=0.2, logy=True
)

In [ ]:
(mean_plot * noise_plot).opts(width=800, height=400)

## Heatmap

In [ ]:
bins = np.geomspace(
    filtered_cells["GFP-PENTA"].min(), filtered_cells["GFP-PENTA"].max(), 100
)

In [ ]:
%%time
channel = "GFP-PENTA"
heatmap = filtered_cells.groupby(["row", "t"]).apply(
    lambda x: pd.Series(np.histogram(x[channel], bins=bins)[0], index=bins[:-1])
)
heatmap.columns.name = channel
heatmap = xr.Dataset({row: heatmap.loc[row].T for row in heatmap.index.levels[0]})

In [ ]:
heatmap.A.hvplot.quadmesh(cmap="blues", logy=True, logz=True, clim=(1, 1e4))

In [ ]:
hv.Layout(
    [
        heatmap[row].hvplot.quadmesh(
            cmap="blues",
            logy=True,
            logz=True,
            clim=(1, 1e4),
        )
        for row in ("A", "B", "C", "D", "E", "F")
    ]
).cols(1)

In [ ]:
%%time
channel = "GFP-PENTA"
heatmap_pos = filtered_cells.groupby(["pos", "t"]).apply(
    lambda x: pd.Series(np.histogram(x[channel], bins=bins)[0], index=bins[:-1])
)
heatmap_pos.columns.name = channel
heatmap_pos = xr.concat(
    [xr.DataArray(heatmap_pos.loc[pos].T) for pos in heatmap_pos.index.levels[0]],
    heatmap_pos.index.levels[0],
)

In [ ]:
grid_df[grid_df["row"] == "D"]

In [ ]:
hv.Layout(
    [
        heatmap_pos.sel(pos=pos).hvplot.quadmesh(
            cmap="blues",
            logy=True,
            logz=True,
            clim=(1, 1e4),
        )
        for pos in range(60, 65)
    ]
).cols(1)